In [ ]:
# Importing Required Libraries
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',None)
import selenium
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import os
import requests
import warnings
warnings.filterwarnings('ignore')
import time
from selenium.common.exceptions import NoSuchElementException

# Instructions

     • All questions are compulsory.
     • In each of the questions you have to automate the process. You do not have to click on any button, click
       any clickable element, enter keywords in search boxes manually. Each process has to be performed via
       coding.
     • Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code
       from beginning for Q2.
     • You may use any web scraping library and tools.
     • The question can be attempted in various ways; the correctness of question depends on the output.
     • If you encounter any Null values during scraping, you may replace it by hyphen.

## Q1.
          Write a python program which searches all the product under a particular product from www.amazon.in.
 
          The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’.
 
          Then search for guitars.


## Q2.
          In the above question, now scrape the following details of each product listed in first 3 pages of your
          search results and save it in a data frame and csv.
 
          In case if any product has less than 3 pages in search
          results then scrape all the products available under that product name.
 
          Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery",   
          "Availability" and “Product URL”.
 
         In case, if any of the details are missing for any of the product then replace it by “-“.

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')   # Creating Instance Of WebDriver
def scraper1_2(u,p):                            # Defining Function With Two Parameters(URL And Product Name)
    driver.get(u)                               # Loading Webpage 
    time.sleep(10)                              # Assigning 10 seconds Of Sleep Time To Get The Page Loaded Properly
    driver.maximize_window()                    # Maximizing Browser Window
    time.sleep(5)
    # Sending Input To The Search Field And Clicking On Search Button
    driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input").send_keys(p)
    driver.find_element_by_xpath("/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input").click()
    time.sleep(10)                              # Assigning 10 seconds Of Sleep Time To Get The Page Loaded Properly
    
    # Creating A DataFrame With Required Columns To Store Scraped Information
    Amazon_Product_Detail = pd.DataFrame([], columns=["Name of the Product", "Brand Name",  "Price", "Return/Exchange",
                                                     "Expected Delivery", "Availability", "Product URL"])
    # Logic To Fetch All The Required Information From The Website
    for page in range(3):
        time.sleep(10)
        items = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        for i in items:
            values = []
            i.click()
            try:
                WebDriverWait(driver, 30).until(EC.number_of_windows_to_be(2))
            except selenium.common.exceptions.TimeoutException:
                time.sleep(10)
                continue
            driver.switch_to.window(driver.window_handles[1])
            # Fetching Name Of Product If Available
            try:
                productName = driver.find_element(By.XPATH, '//div[@id="titleSection"]').text
                if productName:
                    values.append(productName)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching Name Of Brand If AVailable
            try:
                brandName = driver.find_element(By.XPATH, '//*[@id="productOverview_feature_div"]/div/table/tbody/tr[1]/td[2]/span').text
                if brandName:
                    values.append(brandName)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching Price Of Product If Available
            try:
                price = driver.find_element(By.XPATH, '//*[@id="corePrice_feature_div"]/div/span').text.split('₹')[1].split('\n')[0]
                if price:
                    values.append(price)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching If Item is Available For Return Or Exchange
            try:
                returnPolicy = driver.find_element(By.XPATH, '//div[@id="RETURNS_POLICY"]').text
                if returnPolicy:
                    values.append("Available - "+returnPolicy)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching Expected Delivery Date If Available
            try:
                delivery = driver.find_element(By.XPATH, '//div[@id="mir-layout-DELIVERY_BLOCK"]//b').text
                if delivery:
                    values.append(delivery)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching Availability Of Product
            try:
                available = driver.find_element(By.XPATH, '//div[@id="availability"]/span').text
                if available:
                    values.append(available)
                else:
                    raise
            except Exception as e:
                values.append("-")
            # Fetching URL Of The Product Page
            try:
                productUrl = driver.current_url
                if productUrl:
                    values.append(productUrl)
                else:
                    raise
            except Exception as e:
                values.append("-")
                
            # Closing The Product Tab
            driver.close()
            
            # Switching to Previous Tab
            driver.switch_to.window(driver.window_handles[0])
            
            # Adding Scraped Information To The DataFrame
            Amazon_Product_Detail.loc[len(Amazon_Product_Detail)] = values
            
        # Clicking On Next Button
        try:
            driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
        except Exception:
            break    
    
    # Printing DataFrame
    print(Amazon_Product_Detail)
    # Saving DataFrame Into A CSV File
    Amazon_Product_Detail.to_csv("Amazon_Product.csv", index=False)
    
# Taking Desired Product As Input From User
product = input("Enter the Product Of Your Choice : ")
url = "https://www.amazon.in/"             # URL Of Website
scraper1_2(url, product)                   # Calling Function And Passing URL And User Input As Arguments

## Q3.
           Write a python program to access the search bar and search button on images.google.com and

           scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")            # Creating Instance Of Webdriver
def scraper3(a, c):                                      # Defining function with url and product name as parameters
    driver.get(a)                                        # Loading URL Of Website
    driver.maximize_window()                             # Maximizing Browser Window
    time.sleep(5)                                        # Making driver sleep for 5 seconds

    image_dict = {}                                      # Creating an empty Dictionary
    os.makedirs('GoogleImages', exist_ok=True)           # Creating Folder "GoogleImages" In Current Working Directory To Store Downloaded Images
                                                         # If Folder already exists It Will be used to store the downloaded Images  
    # Logic to fetch images related to each keyword
    for keyword in images:
        search_field = driver.find_element(By.XPATH, '//input[@title="Search"]')
        search_field.send_keys(keyword)
        search_field.send_keys(Keys.ENTER)
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        items = driver.find_elements(By.XPATH, '//div[@data-ri]/a[1]/div/img[contains(@src,"http")]')
        for item in items:
            try:
                imageLink = item.get_attribute("src")
            except Exception as e:
                print("Unable to scrap image name and image link --> " + str(e))

            if (imageLink is not None) and (imageLink[0:4] == 'http'):
                if keyword in image_dict.keys():
                    image_dict[keyword].append(imageLink)
                else:
                    image_dict[keyword] = [imageLink]

        search_field = driver.find_element(By.XPATH, '//input[@title="Search"]')
        search_field.clear()
    # Logic to write the downloaded images to the created folder and naming the images
    for key, value in image_dict.items():
        count = 1
        for imgLink in value:
            try:
                if count > 10:
                    break
                response = requests.get(imgLink)
                img = open('GoogleImages/' + key + '_{}.jpeg'.format(count), "wb")
                img.write(response.content)
                print("Downloaded Image {} of 10 of ".format(count) + key)
                count += 1
            except Exception as e:
                continue


url = "https://images.google.com/"                                     # URL Of Website
images = ['Fruits', 'Cars', 'Machine Learning', 'Guitar', 'Cakes']     # List Of Items For Which Images are to be downloaded
scraper3(url, images)                                                  # calling function and passing url and list of items as arguments
        

## Q4. 

      Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
      www.flipkart.com and
      Scrape following details for all the search results displayed on 1st page.
      
      Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary 
      Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. 
      
      Incase if any of the details is missing then replace it by “- “.
      
      Save your results in a dataframe and CSV.
      

In [ ]:
driver=webdriver.Chrome('chromedriver.exe')           # Creating Instance Of Webdriver
def scraper4(c,phone):                                # Defining The Function With URL And Smartphone Name As Parameters
    driver.get(c)                                     # Loading Website
    driver.maximize_window()                          # Maximizing The Chrome Browser Window
    time.sleep(5)                                     # Making Driver To Wait For 5 Seconds
    popup=driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()       # Closing PopUp Window 
    # Finding Search Field Using XPATH And Passing User Input Into It
    search_field=driver.find_element_by_class_name("_3704LK").send_keys(phone) 
    # Finding Search Button Using XPATH And Clicking On it
    search_button=driver.find_element_by_class_name("L0Z3Pu").click()        
    time.sleep(5)                                     # Making Driver To Wait For 5 Seconds 
    #Fetching the url_tags of all the smartphones
    url_tags=driver.find_elements_by_xpath(('//div[@class="_2kHMtA"]/a'))
    urls=[]
    # Creating DataFrame To Store The Outcome
    SPhone_Df=pd.DataFrame([],columns=['Brand Name',"SmartPhone's Name","SmartPhone's Colour",
                                       "SmartPhone's RAM","SmartPhone's ROM",'Display Size',
                                       'Primary Camera','Secondary Camera',"SmartPhone's Battery",
                                       "SmartPhone's Price",'URL Of The Product'])
    # Fetching urls of all the smartphones
    for u in url_tags:
        urls.append(u.get_attribute('href'))
    # Fetching All the Required Information 
    for url in urls:
        # Loading The Urls Of Each Smartphone One By One
        driver.get(url)
        time.sleep(2)
        try:
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')))
        except Exception:
            driver.refresh()
            WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')))
        # Clicking On Read More Button
        read_more_button=driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
        time.sleep(2)
        # fetching brand of smartphone
        try:
            brand_name=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split(' ')[0]
        except Exception:
            brand_name='-'
        # fetching name of smartphone
        try:
            sphone_name=driver.find_element_by_xpath('//td[text()="Model Name"]/following-sibling::td//li').text
        except Exception:
            sphone_name='-'
        # fetching color of smartphone
        try:
            sphone_color=driver.find_element_by_xpath('//td[text()="Color"]/following-sibling::td//li').text
        except Exception:
            sphone_color='-'
        # fetching RAM of smartphone
        try:
            sphone_ram=driver.find_element_by_xpath('//td[text()="RAM"]/following-sibling::td//li').text
        except Exception:
            sphone_ram='-'
        # Fetching ROM of smartphone
        try:
            sphone_rom=driver.find_element_by_xpath('//td[text()="Internal Storage"]/following-sibling::td//li').text
        except Exception:
            sphone_rom='-'
        #Fetching Display Size Of Smartphone
        try:
            display_size=driver.find_element_by_xpath('//td[text()="Display Size"]/following-sibling::td//li').text
        except Exception:
            display_size='-'
        # Fetching Primary Camera Specifications
        try:
            primary_cam=driver.find_element_by_xpath('//td[text()="Primary Camera"]/following-sibling::td//li').text
        except Exception:
            primary_cam='-'
        # Fetching Secondary Camera specifications        
        try:
            sec_cam=driver.find_element_by_xpath('//td[text()="Secondary Camera"]/following-sibling::td//li').text
        except NoSuchElementException as e:
            try:
                sec_cam=driver.find_element_by_xpath('//td[text()="Secondary Camera Features"]/following-sibling::td//li').text
            except NoSuchElementException as e:
                sec_cam="-"
        # fetching battery capacity of smartphone
        try:
            sphone_battery=driver.find_element_by_xpath('//td[text()="Battery Capacity"]/following-sibling::td//li').text
        except Exception:
            sphone_battery='-'
        # fetching price of smartphone
        try:
            sphone_price=driver.find_element_by_xpath('//div[@class="_25b18c"]/div').text
        except Exception:
            sphone_price='-'
        # URL of the page on which all the information about the smartphone is present
        product_url=url
        # List to store all the information gathered above 
        Values=[brand_name,sphone_name,sphone_color,sphone_ram,sphone_rom,display_size,primary_cam,
                sec_cam,sphone_battery,sphone_price,product_url]
        # Passing Gathered information into DataFrame
        SPhone_Df.loc[len(SPhone_Df)]=Values
    # Printing Dataframe
    print(SPhone_Df)                                                                            
    # Saving DataFrame In CSV Format
    SPhone_Df.to_csv('Products.csv',index=False)
# Taking User Input For Smartphone Choice 
sphone=input("Enter the Smartphone You want to search for : ")
url='https://www.flipkart.com/'        # URL of Website
scraper4(url,sphone)                   # Calling Function And Passing URL And User Input As Arguments

## Q5.
         Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
         maps.

In [ ]:
driver=webdriver.Chrome('chromedriver.exe')       # Creating Instance Of Webdriver
def scraper5(d,c):                                # Defining Function
    driver.get(d)                                 # loading Webpage
    driver.maximize_window()                      # Maximizing Chrome Window
    time.sleep(5)                                 # Making driver to wait for 5 Seconds
    
    search_field=driver.find_element_by_id('searchboxinput').send_keys(c)     # Finding Search Field And Sending Input
    search_button=driver.find_element_by_id('searchbox-searchbutton').click() # Finding Search Button And Clicking On It
    time.sleep(5)                                                             # Making driver to wait for 5 Seconds
    
    # Extracting City URL From Webpage
    city_url=driver.find_element_by_xpath('//a[@id="gb_70"]').get_attribute('href')
    
    # Using Regex Methods to Extract Longitude And Latitude Of Entered City
    city_longitude=city_url.split('%2F%40')[1].split('%2C1')[0].split('%2C')[0]
    city_latitude=city_url.split('%2F%40')[1].split('%2C1')[0].split('%2C')[1]
    
    # Printing Extracted Desired Outcomes
    print(f"Longitude Of {c.capitalize()} = {city_longitude}, Latitude Of {c.capitalize()} = {city_latitude}")

url='https://www.google.com/maps/@20.9880135,82.7525294,5z'   # URL Of The Google Maps Website
city=input("Enter City Of Your Choice :")                     # Taking User Input As City
scraper5(url,city)                                            # Passing URL And City As Parameters To The Function

## Q6.
        Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
        from trak.in

In [ ]:
driver=webdriver.Chrome('chromedriver.exe')       # creating instance of webdriver
def scraper6(a):                                  # defining a function with url as a parameter
    driver.get(a)                                 # loading webpage using url passed as an argument to the function
    driver.maximize_window()                      # maximizing browser window
    time.sleep(5)                                 # making driver wait for 5 seconds
    # creating a dataframe to store all the required information
    Investment_Details = pd.DataFrame([], columns=["Sr.No.", "Date", "Startup Name", "Industry/Vertical", "Sub-Vertical",
                                               "City/Location", "Investor's Name", "Investment Type", "Amount (In USD)"])
    
    # fetching funding-deals page url from homepage to gather the required information
    funding_deals_url=driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
    driver.get(funding_deals_url)
    time.sleep(5)
    
    # fetching month tags using xpath
    month_names = driver.find_elements(By.XPATH, '//h2[contains(@class,"tablepress-table-name")]')
    # fetching all the information related to funding deals of second quarter
    for month in month_names:
        if month.text in ['March, 2021', 'February, 2021', 'January, 2021']:
            table_id = month.get_attribute("class").split(' ')[1].split('-')[4]
            xpath_string = '//table[@id="tablepress-' + table_id + '"]//tbody/tr'
            values = [i.text for i in driver.find_elements(By.XPATH, xpath_string+'/td')]
            for i in range(0, len(values), 9):
                Investment_Details.loc[len(Investment_Details)] = values[i:i + 9]
    # dropping "Sr.No." Columns from dataframe
    Investment_Details = Investment_Details.drop('Sr.No.',axis=1)
    # printing dataframe
    print(Investment_Details)
# URL of website
url = "https://trak.in"
scraper6(url)              # calling function with url as an argument

## Q7.
         Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")          # Creating Instance Of Webdriver
def scraper7(a):                                       # creating function with url as a parameter
    driver.get(a)                                      # Loading the website
    driver.maximize_window()                           # maximizing browser window
    time.sleep(5)                                      # making driver to sleep for 5 seconds
    # creating a dataframe to store all the required information after scraping
    Product_df = pd.DataFrame([], columns=["Laptop's Name", "Laptop's Price", "Laptop's OS", "Display Size(In Inches)",
                                           "Processor's Name", "Laptop's HDD",
                                           "Laptop's RAM", "Laptop's Weight(In Kgs)", "Laptop's Dimensions(In MM)",
                                           "Laptop's GPU"])
    # clicking on "Top10" Link
    Top10_link = driver.find_element(By.XPATH, '//a[text()="Top 10"]').click()
    time.sleep(2)
    # clicking on "Laptops" option 
    laptops_button = driver.find_element(By.XPATH, '//div[@class="categoty_list"]/button/img[@title="Laptops"]')
    ActionChains(driver).move_to_element(laptops_button).click().perform()
    time.sleep(2)
    # clicking on "Best Gaming Laptops in India" link on the page
    driver.find_element(By.XPATH, '//a[text()="Best Gaming Laptops in India"]').click()
    
    #Logic to click on "Full specs" option related to every product and then scrape the required information
    items = driver.find_elements_by_xpath('//span[text()="Full specs"]')
    for item in items:
        driver.execute_script("window.open('%s', '_blank')" % item.get_attribute('data-href'))
        try:
            WebDriverWait(driver, 20)
        except selenium.common.exceptions.TimeoutException:
            time.sleep(10)
            continue
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3)
        
        # fetching model name of laptop
        try:
            productName = driver.find_element_by_xpath('//td[text()="Model name"]/following-sibling::td[2]').text
        except Exception:
            productName = '-'
        
        # fetching Operating System of laptop
        try:
            osName = driver.find_element_by_xpath('//td[text()="operating system (with version)"]/following-sibling::td[2]').text
        except Exception:
            osName = '-'
        
        # fetching size of display of laptop
        try:
            displaySize = driver.find_element_by_xpath('//td[text()="display size (in inches)"]/following-sibling::td[2]').text
        except Exception:
            displaySize = '-'
        
        # fetching name of processor of laptop
        try:
            processorName = driver.find_element_by_xpath('//td[text()="processor model name"]/following-sibling::td[2]').text
        except Exception:
            processorName = '-'
        
        # fetching storage type and storage capacity of laptop
        try:
            hddType = driver.find_element_by_xpath('//td[text()="Storage drive type"]/following-sibling::td[2]').text
            hddCapacity = driver.find_element_by_xpath('//td[text()="Storage drive capacity"]/following-sibling::td[2]').text
            Storage = hddCapacity+" "+hddType
        except Exception:
            Storage = '-'
        
        # fecthing RAM type and capacity of laptop
        try:
            ramCapacity = driver.find_element_by_xpath('//td[text()="ram included (in gb)"]/following-sibling::td[2]').text
            ramType = driver.find_element_by_xpath('//td[text()="ram type"]/following-sibling::td[2]').text
            ram = ramCapacity +" "+ramType
        except Exception:
            ram = '-'
        
        # fetching weight of laptop
        try:
            weight = driver.find_element_by_xpath('//td[text()="laptop weight (in kgs)"]/following-sibling::td[2]').text
        except Exception:
            weight = '-'
        
        # fetching dimensions of laptop
        try:
            dimensions = driver.find_element_by_xpath('//td[text()="laptop dimension (in mm)"]/following-sibling::td[2]').text
        except Exception:
            dimensions = '-'
        
        # fetching Graphics Processor Name
        try:
            gpu = driver.find_element_by_xpath('//td[text()="graphics processor"]/following-sibling::td[2]').text
        except Exception:
            gpu = '-'
        
        # fetching price of laptop
        try:
            price = driver.find_element_by_xpath('//span[@class="price last-child"]').text
        except Exception:
            price = '-'
        
        # storing all the fetched values in a list
        Values = [productName, price, osName, displaySize, processorName, Storage, ram, weight, dimensions, gpu]
        # closing the tab
        driver.close()
        # switching to previous tab
        driver.switch_to.window(driver.window_handles[0])
        # passing values to Dataframe 
        Product_df.loc[len(Product_df)] = Values
   
    # printing dataframe
    print(Product_df)

url = "https://www.digit.in/"          # URL of website
scraper7(url)                          # calling function and passing URL as an argument

## Q8.
        Write a python program to scrape the details for all billionaires from www.forbes.com.
        Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")          # creatig instance of webdriver
def scraper8(a):                                       # creating function with url as a parameter
    driver.get(a)                                      # loading website
    driver.maximize_window()                           # maximizing browser window
    time.sleep(5)                                      # making driver sleep for 5 seconds
    # creating a dataframe to store all the scraped information
    Billionaires_df = pd.DataFrame([], columns=["Rank", "Name", "Net Worth", "Age", "Citizenship", "Source", "Industry"])
    
    # logic to fetch the required information from website
    count = 0
    while True:
        count += 1
        time.sleep(5)
        # logic for closing the popup if it occurs
        if count == 2:
            try:
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@class="tp-close tp-active"]'))).click()
            except Exception:
                pass
        billionaire_rows = [item.text for item in driver.find_elements(By.XPATH, '//div[@class="table-row-group__container"]/div[@role="row"]')]
        # iterating table row content and splitting the values as per each of the columns
        for item in billionaire_rows:
            splitted_row_values = item.split('\n')
            if len(splitted_row_values) > 7:
                splitted_row_values.remove('★')
            splitted_row_values[0] = splitted_row_values[0].split('.')[0]
            Billionaires_df.loc[len(Billionaires_df)] = splitted_row_values
            
        # clicking on "NEXT" button
        try:
            driver.find_element(By.XPATH, '//button[@class="pagination-btn pagination-btn--next "]').click()
        except Exception:
            break
    # printing dataframe
    print(Billionaires_df)

url = "https://www.forbes.com/billionaires/"               # URL of website
scraper8(url)                                              # calling function with url as an argument

## Q9. 
          Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
          from any YouTube Video

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")      # creating instance of webdriver
def scraper9(u):                                   # defining a function with url as a parameter
    driver.get(u)                                  # loading the website
    driver.maximize_window()                       # maximizing browser window
    time.sleep(5)                                  # making the driver to sleep for 5 seconds
    # creating a dataframe to store all the scraped information
    Comment_Info=pd.DataFrame([], columns=['Comment', 'Time of Comment', 'Likes on Comment'])
    # logic to check if the title of the video is available to fetch
    try:
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="container"]/h1/yt-formatted-string')))
    except Exception:             
        driver.refresh()
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="container"]/h1/yt-formatted-string')))
    
    # Logic to scrape the required values
    # driver will pause for 2 seconds initially after scrolling to load the comments
    pause_time_while_scrolling = 2
    # driver will scroll for 25 times
    cycles = 25

    html = driver.find_element(By.TAG_NAME, 'html')

    try:
        html.send_keys(Keys.PAGE_DOWN)
    except Exception:
        html.send_keys(Keys.PAGE_DOWN)

    time.sleep(pause_time_while_scrolling*3)
    # fetching the title of the video
    title = driver.find_element(By.XPATH, '//*[@id="container"]/h1/yt-formatted-string').text.split('|')
    print("Scrapping Details for - " + title[0]+" | "+title[1])
    # scrolling for each cycle and pausing for certain time
    for i in range(cycles):
        html.send_keys(Keys.END)
        time.sleep(pause_time_while_scrolling)
    # fetching all comment sections
    comments_section= driver.find_elements(By.XPATH, '//div[@id="contents"]/ytd-comment-thread-renderer')
    # fetching all the required information from comments_section
    for i in range(1, len(comments_section)+1):
        # fetching comment text if available
        try:
            comments_text = driver.find_element(By.XPATH, '//div[@id="contents"]/ytd-comment-thread-renderer[{i}]//yt-formatted-string[@id="content-text"]'.format(i=i)).text.replace('\n',' ')
        except Exception:
            comments_text = "-"
        # fetching comment time if available
        try:
            comment_time = driver.find_element(By.XPATH, '//div[@id="contents"]/ytd-comment-thread-renderer[{i}]//yt-formatted-string[contains(@class,"published-time-text")]'.format(i=i)).text
        except Exception:
            comment_time = "-"
        # fetching no of likes on comment if available
        try:
            likes_count = driver.find_element(By.XPATH, '//div[@id="contents"]/ytd-comment-thread-renderer[{i}]//span[@id="vote-count-middle"]'.format(i=i)).text
            if likes_count == "":
                likes_count = 0
        except Exception:
            likes_count = "-"
        # storing all the scraped values in a list
        Values = [comments_text, comment_time, likes_count]
        # adding all the values to the dataframe
        Comment_Info.loc[len(Comment_Info)] = Values
    # printing dataframe
    print(Comment_Info)
# URL of the youtube video
vid_url="https://www.youtube.com/watch?v=zR5-HbFW6hc"
scraper9(vid_url)   # calling funtion with url as an argument

## Q10.
        Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
        “London” location. 
        You have to scrape hostel name, distance from city centre, ratings, total reviews,
        overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
driver = webdriver.Chrome("chromedriver.exe")    # creating instance of webdriver
def scraper10(a):                                # defining a function with url of the website as a parameter
    driver.get(a)                                # loading website
    driver.maximize_window()                     # maximizing browser window
    time.sleep(5)                                # making driver wait for 5 seconds after loading the website
    
    # creating a dataframe to store all the scraped information 
    Hostel_Details = pd.DataFrame([], columns=["Hostel Name", "Distance from City Centre", "Ratings", "Total Reviews", "Overall Review",
                                               "Privates from price", "Dorms from price", "Facilities", "Property Description"])
    # logic to scrape all the required information
    page = 0
    div_count = 0
    while True:
        page += 1
        time.sleep(10)
        properties = driver.find_elements(By.XPATH, '//div[@class="message-container"]/../div[@class="property-card"]//a[@class="view-button"]')
        for i in range(0, len(properties)):
            if page > 1:
                div_count = 2
            else:
                div_count = 3
            # fetching name of hostel if available
            try:
                hostelName = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//h2[contains(@class,"title-6")]'.format(no=i+div_count)).text
            except Exception:
                hostelName = "-"
            # fetching distance of hostel from city centre if available
            try:
                distanceCityCentre = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[@class="property"]//div[@class="title-row"]//div[@class="subtitle body-3"]/a/span[@class="description"]'.format(no=i+div_count)).text.split('-')[1].split(' ')[0]
            except Exception:
                distanceCityCentre = "-"
            # fetching ratings of hostel if available
            try:
                rating = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"score")]'.format(no=i+div_count)).text
            except Exception:
                rating = "-"
            # fetching total number of reviews if available
            try:
                totalReview = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"summary ")]//div[@class="reviews"]'.format(no=i+div_count)).text.split('Total')[0]
            except Exception:
                totalReview = "-"
            # fetching overall review if available
            try:
                overallReview = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"summary ")]//div[@class="keyword"]'.format(no=i+div_count)).text
            except Exception:
                overallReview = "-"
            # fetching price of private rooms if available
            try:
                privatesPrice = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"prices-col")]//a[@class="prices"]/div[1]/div'.format(no=i+div_count)).text.split(' ')[1]
            except Exception:
                try:
                    privatesPrice = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"prices-col")]//a[@class="prices"]/div[1]/div'.format(no=i + div_count)).text
                except Exception:
                    privatesPrice = "-"
            # fetching price of dorm if available
            try:
                dormPrice = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"prices-col")]//a[@class="prices"]/div[2]/div'.format(no=i+div_count)).text.split(' ')[1]
            except Exception:
                try:
                    dormPrice = driver.find_element(By.XPATH, '//div[@class="message-container"]/../div[{no}][@class="property-card"]//div[contains(@class,"prices-col")]//a[@class="prices"]/div[2]/div'.format(no=i + div_count)).text
                except Exception:
                    dormPrice = "-"
            # clicking on property link
            try:
                properties[i].click()
            except Exception:
                pass
            
            try:
                ActionChains(driver).key_down(Keys.CONTROL).click(properties[i]).key_up(Keys.CONTROL).perform()
                try:
                    WebDriverWait(driver, 30).until(EC.number_of_windows_to_be(2))
                except selenium.common.exceptions.TimeoutException:
                    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                    continue
                driver.switch_to.window(driver.window_handles[1])
            except Exception:
                print("Unable to click 'View' Button")
            # fetching all the facilities if available
            try:
                facility = driver.find_elements(By.XPATH, '//div[@class="facilities-container"]//ul[@class="facility-sections"]//ul[@class="facilities"]/li')
                facilities = [f.text for f in facility]
            except Exception:
                facilities = "-"
            # fetching description of property if available
            try:
                propertyDescription = driver.find_element(By.XPATH, '//div[@class="flex-80"]//div[@class="description-container"]/div/div[2]').text.replace('\n','')
            except Exception:
                propertyDescription = "-"
            # closing new tab and switching to previous tab
            try:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            except Exception as e:
                print("Driver Session Expired - Unable to switch driver window")
            # passing scraped information to the dataframe
            Hostel_Details.loc[len(Hostel_Details)] = [hostelName, distanceCityCentre, rating, totalReview, overallReview, privatesPrice, dormPrice, ', '.join(facilities), propertyDescription]
        # clicking on "NEXT" button
        try:
            next_btn = driver.find_element(By.XPATH, '//div[@class="pagination-item pagination-next"]')
            driver.execute_script('arguments[0].click()', next_btn)
        except Exception as e:
            break
    # printing dataframe
    print(Hostel_Details)
    # converting dataframe into a CSV file
    Hostel_Details.to_csv("Hostelworld.csv", index=False)

# getting "City" as an input from user
city = input("Please Enter the City in which you are looking for hostels :")
url = "https://www.hostelworld.com/s?q={}&type=city&id=3&guests=1".format(city)    # URL of the website
scraper10(url)                                                                     # calling function with url as an argument